### Installs and Imports

In [5]:
!pip install -q cassandra-driver
!pip install -q cassio
!pip install -q langchain
!pip install -q typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install -q pydantic==1.10.11
!pip install -q flask-sqlalchemy
!pip install -q unstructured


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
#!pip install -q fiddler-client

In [11]:
! pip install -q --progress-bar off \
    "git+https://github.com/hemidactylus/langchain@updated-full-preview--lab#egg=langchain&subdirectory=libs/langchain" \
    "cassio>=0.1.1" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2" 
exit()

ERROR: Could not find a version that satisfies the requirement tensorflow-cpu==2.12.0 (from versions: none)
ERROR: No matching distribution found for tensorflow-cpu==2.12.0

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os
import pandas as pd
import fiddler as fdl

In [2]:
fdl.__version__

'2.5.0.dev1'

In [3]:
cassandra.__version__

'3.29.0'

### Connect to DataStax

In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'}

ASTRA_DB_APPLICATION_TOKEN = 'ADD Your Token'
#print("TOKEN: " + ASTRA_DB_APPLICATION_TOKEN)

auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'

In [46]:
os.environ['OPENAI_API_KEY']='ADD Your OpenAI Key'

In [14]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader, DirectoryLoader, DataFrameLoader

In [15]:
from langchain.vectorstores.cassandra import Cassandra

In [16]:
import os
import openai

In [31]:
os.environ['OPENAI_API_TYPE'] = 'open_ai'
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0)
myEmbedding = OpenAIEmbeddings()
print('LLM+embeddings from OpenAI')

LLM+embeddings from OpenAI


In [32]:
table_name = 'fiddler_doc_snippets_' + llmProvider

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=myEmbedding,
    text_splitter=CharacterTextSplitter(
        chunk_size=5000,
        chunk_overlap=0,
    ),
    vectorstore_kwargs={
        'session': session,
        'keyspace': 'fiddlerai',
        'table_name': table_name,
    },
)

In [33]:
df = pd.read_csv('documentation_data/vector_index_feed_24.1.csv')
df

,text
0,"---\ntitle: ""fdl.FiddlerApi""\nslug: ""client-se..."
1,"slug: ""client-setup"" url=URL,\n org_id=..."
2,"---\ntitle: ""Customer Churn Prediction""\nslug:..."
3,"slug: ""customer-churn-prediction"" ""https://fi..."
4,"slug: ""customer-churn-prediction"" ca-1.png"",\n..."
...,...
4153,"Currently, only the following fields in [fdl.M..."
4154,AI has been in the limelight thanks to ‌recent...
4155,LLM means large language model. A large langu...
4156,"The term generative AI, or GenAI, also is clos..."


In [34]:
loader = DataFrameLoader(df, page_content_column="text")
len(loader.load())

4158

In [36]:
index = index_creator.from_loaders([loader])
index

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-VifEdlDlDNEVzwTxlrODi3pU on tokens per min (TPM): Limit 1000000, Used 612185, Requested 757379. Please try again in 22.173s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-VifEdlDlDNEVzwTxlrODi3pU on tokens per min (TPM): Limit 1000000, Used 522028, Requested 757379. Please try again in 16.764s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-VifEdlDlDNEVzwTxlrODi3pU on tokens pe

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.cassandra.Cassandra object at 0x12cee8ac0>)

In [44]:
cqlSelect = f'SELECT * FROM fiddlerai.{table_name} LIMIT 2;'  # (Not a production-optimized query ...)
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    # depending on the cassIO version, the underlying Cassandra table can have different structure ...
    try:
        # you are using the new cassIO 0.1.0+ : congratulations :)
        print(f'    row_id:            {row.row_id}')
        print(f'    vector:            {str(row.vector)[:64]} ...')
        print(f'    body_blob:         {row.body_blob} ...')
        print(f'    metadata_s:        {row.metadata_s}')        
    except AttributeError:
        # Please upgrade your cassIO to the latest version ...
        print(f'    document_id:      {row.document_id}')
        print(f'    embedding_vector: {str(row.embedding_vector)[:64]} ...')
        print(f'    document:         {row.document[:64]} ...')
        print(f'    metadata_blob:    {row.metadata_blob}')

print('\n...')


Row 0:
    row_id:            c37e08bf1c8c4756b3780e7d3c000073
    vector:            [-0.015756040811538696, 0.014646461233496666, 0.0119418594986200 ...
    body_blob:         BlogLink:https://www.fiddler.ai/blog/monitoring-natural-language-processing-and-computer-vision-models-part-3 Content: In the previous parts of this blog series about monitoring models with unstructured data, we covered Fiddler’s approach to monitoring high-dimensional vectors, and how this method can be applied to computer vision use cases.  In part 3, we will discuss why monitoring natural language processing (NLP) models is important, and show how Fiddler empowers data scientists and ML practitioners with NLP model monitoring. We will present a multi-class classification example which is built using the 20 Newsgroups dataset and use text embedding generated by OpenAI.  Over the past decade, NLP has become an important technology adopted by data-driven enterprises. Investments will continue to pour into NLP 

In [ ]:
query = "What is Fiddler?"
index.query(query, llm=llm)

In [ ]:
createTableSQL = """CREATE TABLE fiddler_chatbot_history (
    row_id text PRIMARY KEY,
    response text,
    response_vector vector<float, 1536>,
    source_docs text,
    source_docs_vector vector<float, 1536>,
    question text,
    question_vector vector<float, 1536>,
    comment text,
    feedback int,
    metadata_s map<text, text>,
    ts timestamp)"""
# ) WITH additional_write_policy = '99p'
#     AND bloom_filter_fp_chance = 0.01
#     AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
#     AND comment = ''
#     AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
#     AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
#     AND crc_check_chance = 1.0
#     AND default_time_to_live = 0
#     AND gc_grace_seconds = 864000
#     AND max_index_interval = 2048
#     AND memtable_flush_period_in_ms = 0
#     AND min_index_interval = 128
#     AND read_repair = 'BLOCKING'
#     AND speculative_retry = '99p';"""

In [ ]:
session.execute(createTableSQL)

In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"
response = openai.Embedding.create(model=EMBEDDING_MODEL, input='How are you doing')
response

In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [ ]:
rows = session.execute('SELECT * from squad')
    
df_baseline = rows._current_rows
df_baseline

In [ ]:
column_types = df_baseline.dtypes

print(column_types)

In [ ]:
df_baseline['answers'] = df_baseline['answers'].apply(lambda x : str(x))